# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

''' Para rodar o chrome em 2º plano
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
nav = webdriver.Chrome(options=chrome_options)
'''

# Abrir navegador
nav = webdriver.Chrome()

# Passo 1: Capturar a cotação do dolar
nav.get('https://www.google.com/')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# Passo 2: Capturar a cotação do euro
nav.get('https://www.google.com/')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# Passo 3: Capturar a cotação do ouro
nav.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = nav.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

nav.quit()

4.9345
5.889863610000001
282.62


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [13]:
# Passo 4: Importar a lista de produtos
import pandas as pd 

df = pd.read_excel('Produtos.xlsx')
display(df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [15]:
# Passo 5: Recalcular o preço de cada produto

# Atualizar a cotação
# Comando --> tabela.loc[linha, coluna] = valor
# Exemplo  --> Se o campo "Moeda" for igual a "Dolar", coloque o valor de cotacao_dolar no campo "Cotação"
# Nas linhas onde a coluna "Moeda" == Dolar | "Moeda" == Euro | "Moeda" == Ouro coloca-se a nova cotação
df.loc[df["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
df.loc[df["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
df.loc[df["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Atualizar o preço base reais (preço base original * cotação)
df['Preço Base Reais'] = df['Preço Base Original'] * df['Cotação']
# Atualizar o preço final (preço base reais * margem)
df['Preço Final'] = df['Preço Base Reais'] * df['Margem']

display(df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.934500,4934.450655,1.40,6908.230917
1,Carro Renault,4500.00,Euro,5.889864,26504.386245,2.00,53008.772490
2,Notebook Dell,899.99,Dólar,4.934500,4441.000655,1.70,7549.701113
3,IPhone,799.00,Dólar,4.934500,3942.665500,1.70,6702.531350
4,Carro Fiat,3000.00,Euro,5.889864,17669.590830,1.90,33572.222577
5,Celular Xiaomi,480.48,Dólar,4.934500,2370.928560,2.00,4741.857120
6,Joia 20g,20.00,Ouro,282.620000,5652.400000,1.15,6500.260000


### Agora vamos exportar a nova base de preços atualizada

In [ ]:
df.to_excel('Produtos Novo.xlsx', index=True)